In [1]:
%cd /home/adityasidharta/git/shopee_data_science
%env PROJECT_PATH /home/adityasidharta/git/shopee_data_science

/home/adityasidharta/git/shopee_data_science
env: PROJECT_PATH=/home/adityasidharta/git/shopee_data_science


In [2]:
from utils.envs import *
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import string

In [3]:
beauty_train = pd.read_csv(beauty_train_repo)
beauty_val = pd.read_csv(beauty_val_repo)

fashion_train = pd.read_csv(fashion_train_repo)
fashion_val = pd.read_csv(fashion_val_repo)

mobile_train = pd.read_csv(mobile_train_repo)
mobile_val = pd.read_csv(mobile_val_repo)

In [4]:
print(beauty_train.columns)
print(fashion_train.columns)
print(mobile_train.columns)

Index(['itemid', 'title', 'image_path', 'Colour_group', 'Brand', 'Benefits',
       'Product_texture', 'Skin_type'],
      dtype='object')
Index(['itemid', 'title', 'Pattern', 'Collar Type', 'Sleeves', 'Fashion Trend',
       'Clothing Material', 'image_path'],
      dtype='object')
Index(['itemid', 'title', 'image_path', 'Operating System', 'Features',
       'Network Connections', 'Memory RAM', 'Brand', 'Warranty Period',
       'Storage Capacity', 'Color Family', 'Phone Model', 'Camera',
       'Phone Screen Size'],
      dtype='object')


In [5]:
print(beauty_val.columns)
print(fashion_val.columns)
print(mobile_val.columns)

Index(['itemid', 'title', 'image_path'], dtype='object')
Index(['itemid', 'title', 'image_path'], dtype='object')
Index(['itemid', 'title', 'image_path'], dtype='object')


In [6]:
def text_process(s):
    s = str(s)
    s.translate(str.maketrans('','',string.punctuation))
    s = s.lower()
    return s

In [7]:
feature_encoder = TfidfVectorizer()
beauty_train['title'].values

array(['nyx sex bomb pallete natural palette',
       'etude house precious mineral any cushion pearl aura puff',
       'milani rose powder blush', ..., 'tony moly delight tint mini red',
       'nyx soft matte lip cream smlc shade san paolo',
       'liquid lipstick anastasia beverly hills matte colour'],
      dtype=object)

In [8]:
def create_label(df, test_df, topic):
    result_dict = {}
    feature_df = df[['title']].copy()
    label_df = df.drop(columns=['itemid', 'title', 'image_path']).copy()
    
    feature_df['title'] = feature_df['title'].apply(lambda x: text_process(x))
    feature_array = feature_df['title'].values.tolist()
    feature_encoder = TfidfVectorizer()
    feature_encoder.fit(feature_array)
    feature_attr = feature_encoder.transform(feature_array)
    feature_decomposer = TruncatedSVD(500)
    feature_decomposer.fit(feature_attr)
    feature_attr = feature_decomposer.transform(feature_attr)
    
    test_df['title'] = test_df['title'].apply(lambda x: text_process(x))
    test_array = test_df['title'].values.tolist()
    test_attr = feature_encoder.transform(test_array)
    test_attr = feature_decomposer.transform(test_attr)
    
    train_itemid = df['itemid']
    test_itemid = test_df['itemid']
    
    result_dict['itemid_train_{}'.format(topic)] = train_itemid
    result_dict['itemid_test_{}'.format(topic)] = test_itemid
    result_dict['X_train_{}'.format(topic)] = feature_attr
    result_dict['X_encoder_{}'.format(topic)] = feature_encoder
    result_dict['X_decomposer_{}'.format(topic)] = feature_decomposer
    result_dict['X_test_{}'.format(topic)] = test_attr
    
    for column in label_df.columns:

        label_encoder = OrdinalEncoder(cols=[column], handle_unknown='impute')
        label_encoder.fit(label_df[[column]])
        label_attr = label_encoder.transform(label_df[[column]])

        result_dict['Y_train_{}_{}'.format(topic, column)] = label_attr
        result_dict['Y_encoder_{}_{}'.format(topic, column)] = label_encoder
        result_dict['Y_colname_{}_{}'.format(topic, column)] = label_attr.columns
        
    return result_dict

In [9]:
beauty_dict = create_label(beauty_train, beauty_val, 'beauty')
fashion_dict = create_label(fashion_train, fashion_val, 'fashion')
mobile_dict = create_label(mobile_train, mobile_val, 'mobile')

In [14]:
topic = 'beauty'
column = 'Benefits'

X_train = beauty_dict['X_train_{}'.format(topic)]
Y_train = beauty_dict['Y_train_{}_{}'.format(topic, column)][column].values

train_idx = Y_train != 0
X_train = X_train[train_idx]
Y_train = Y_train[train_idx]

X_dev, X_val, Y_dev, Y_val = train_test_split(X_train, Y_train, test_size = 0.2)

X_test = beauty_dict['X_test_{}'.format(topic)]

dtrain = lgb.Dataset(X_train, label=Y_train)
ddev = lgb.Dataset(X_dev, label=Y_dev)
dval = lgb.Dataset(X_val, label=Y_val)
dtest = lgb.Dataset(X_test)

params = {'objective': 'multiclass',
         'eta': 0.01,
         'max_depth': 6,
         "feature_fraction":0.7,
         "bagging_fraction":0.7,
         "num_leaves": 63,
         'nthread': 6,
         'num_class': Y_train.max() + 1}

bst = lgb.train(params, ddev, num_boost_round=10, valid_sets=[ddev, dval], valid_names=['ddev','dval'], early_stopping_rounds=50)

[1]	ddev's multi_logloss: 2.39357	dval's multi_logloss: 2.39968
Training until validation scores don't improve for 50 rounds.
[2]	ddev's multi_logloss: 2.34186	dval's multi_logloss: 2.35128
[3]	ddev's multi_logloss: 2.29831	dval's multi_logloss: 2.31035
[4]	ddev's multi_logloss: 2.25989	dval's multi_logloss: 2.27476
[5]	ddev's multi_logloss: 2.22429	dval's multi_logloss: 2.24133
[6]	ddev's multi_logloss: 2.191	dval's multi_logloss: 2.21034
[7]	ddev's multi_logloss: 2.16054	dval's multi_logloss: 2.18179
[8]	ddev's multi_logloss: 2.13191	dval's multi_logloss: 2.15481
[9]	ddev's multi_logloss: 2.1047	dval's multi_logloss: 2.12919
[10]	ddev's multi_logloss: 2.07876	dval's multi_logloss: 2.10485
Did not meet early stopping. Best iteration is:
[10]	ddev's multi_logloss: 2.07876	dval's multi_logloss: 2.10485


In [16]:
Y_pred = bst.predict(X_test)
Y_class = Y_pred.argmax(1)
Y_class_df = pd.DataFrame(Y_class, columns = beauty_dict['Y_colname_beauty_Colour_group'])
Y_class_result = beauty_dict['Y_encoder_beauty_Colour_group'].inverse_transform(Y_class_df)
result = pd.DataFrame({
    'id': beauty_dict['itemid_test_beauty'].apply(lambda x : str(x) + '_Colour_group'),
    'tagging': Y_class_result['Colour_group'].values
})

In [22]:
def create_prediction(topic_dict, topic, column_list):
    result_dict = {}
    for column in column_list:
        print('######################################################')
        print(column)
        print('######################################################')
        X_train = topic_dict['X_train_{}'.format(topic)]
        Y_train = topic_dict['Y_train_{}_{}'.format(topic, column)][column].values
        
        train_idx = Y_train != 0
        X_train = X_train[train_idx]
        Y_train = Y_train[train_idx]
        
        X_dev, X_val, Y_dev, Y_val = train_test_split(X_train, Y_train, test_size = 0.1)
        
        X_test = topic_dict['X_test_{}'.format(topic)]
        
        ddev = lgb.Dataset(X_dev, label=Y_dev)
        dval = lgb.Dataset(X_val, label=Y_val)

        params = {'objective': 'multiclass',
                  'eta': 0.01,
                  'max_depth': 6,
                  'num_leaves': 63,
                  "feature_fraction":0.7,
                  "bagging_fraction":0.7,
                  'silent': 1,
                  'nthread': 6,
                  'num_class': Y_train.max() + 1}

        bst = lgb.train(params, ddev, num_boost_round=2000, valid_sets=[ddev, dval], valid_names=['ddev','dval'], early_stopping_rounds=50)
        Y_pred = bst.predict(X_test)
        
        Y_class = Y_pred.argmax(1)
        Y_class_df = pd.DataFrame(Y_class, columns = topic_dict['Y_colname_{}_{}'.format(topic, column)])
        Y_class_result = topic_dict['Y_encoder_{}_{}'.format(topic, column)].inverse_transform(Y_class_df)
        result = pd.DataFrame({
            'id': topic_dict['itemid_test_{}'.format(topic)].apply(lambda x : str(x) + '_{}'.format(column)),
            'tagging': Y_class_result[column].values.astype(int).astype(str)
        })
        result_dict['result_df_{}_{}'.format(topic, column)] = result
    return result_dict
        

In [23]:
beauty_result_dict = create_prediction(beauty_dict, 'beauty', ['Colour_group', 'Brand', 'Benefits',
       'Product_texture', 'Skin_type'])
fashion_result_dict = create_prediction(fashion_dict, 'fashion', ['Pattern', 'Collar Type', 'Sleeves', 'Fashion Trend',
       'Clothing Material'])
mobile_result_dict = create_prediction(mobile_dict, 'mobile', ['Operating System', 'Features',
       'Network Connections', 'Memory RAM', 'Brand', 'Warranty Period',
       'Storage Capacity', 'Color Family', 'Phone Model', 'Camera',
       'Phone Screen Size'])

######################################################
Colour_group
######################################################
[1]	ddev's multi_logloss: 1.50906	dval's multi_logloss: 1.5261
Training until validation scores don't improve for 50 rounds.
[2]	ddev's multi_logloss: 1.49382	dval's multi_logloss: 1.5104
[3]	ddev's multi_logloss: 1.47973	dval's multi_logloss: 1.49619
[4]	ddev's multi_logloss: 1.466	dval's multi_logloss: 1.48217
[5]	ddev's multi_logloss: 1.45277	dval's multi_logloss: 1.46866
[6]	ddev's multi_logloss: 1.44005	dval's multi_logloss: 1.45573
[7]	ddev's multi_logloss: 1.42769	dval's multi_logloss: 1.44329
[8]	ddev's multi_logloss: 1.41581	dval's multi_logloss: 1.43128
[9]	ddev's multi_logloss: 1.40429	dval's multi_logloss: 1.41962
[10]	ddev's multi_logloss: 1.39324	dval's multi_logloss: 1.40849
[11]	ddev's multi_logloss: 1.38236	dval's multi_logloss: 1.39748
[12]	ddev's multi_logloss: 1.37185	dval's multi_logloss: 1.3869
[13]	ddev's multi_logloss: 1.36138	dval's multi_l

[123]	ddev's multi_logloss: 0.808526	dval's multi_logloss: 0.82713
[124]	ddev's multi_logloss: 0.805878	dval's multi_logloss: 0.824562
[125]	ddev's multi_logloss: 0.803309	dval's multi_logloss: 0.822023
[126]	ddev's multi_logloss: 0.800731	dval's multi_logloss: 0.81954
[127]	ddev's multi_logloss: 0.798138	dval's multi_logloss: 0.817013
[128]	ddev's multi_logloss: 0.795628	dval's multi_logloss: 0.814548
[129]	ddev's multi_logloss: 0.79305	dval's multi_logloss: 0.812035
[130]	ddev's multi_logloss: 0.790535	dval's multi_logloss: 0.809586
[131]	ddev's multi_logloss: 0.788023	dval's multi_logloss: 0.807133
[132]	ddev's multi_logloss: 0.785586	dval's multi_logloss: 0.804726
[133]	ddev's multi_logloss: 0.783188	dval's multi_logloss: 0.802377
[134]	ddev's multi_logloss: 0.780793	dval's multi_logloss: 0.800021
[135]	ddev's multi_logloss: 0.778359	dval's multi_logloss: 0.797669
[136]	ddev's multi_logloss: 0.775991	dval's multi_logloss: 0.795392
[137]	ddev's multi_logloss: 0.77362	dval's multi_lo

[244]	ddev's multi_logloss: 0.603118	dval's multi_logloss: 0.629554
[245]	ddev's multi_logloss: 0.601996	dval's multi_logloss: 0.628528
[246]	ddev's multi_logloss: 0.600909	dval's multi_logloss: 0.627524
[247]	ddev's multi_logloss: 0.599863	dval's multi_logloss: 0.626559
[248]	ddev's multi_logloss: 0.598823	dval's multi_logloss: 0.625596
[249]	ddev's multi_logloss: 0.597748	dval's multi_logloss: 0.624601
[250]	ddev's multi_logloss: 0.596643	dval's multi_logloss: 0.623617
[251]	ddev's multi_logloss: 0.595619	dval's multi_logloss: 0.62268
[252]	ddev's multi_logloss: 0.594598	dval's multi_logloss: 0.621713
[253]	ddev's multi_logloss: 0.59358	dval's multi_logloss: 0.620812
[254]	ddev's multi_logloss: 0.592538	dval's multi_logloss: 0.619834
[255]	ddev's multi_logloss: 0.59153	dval's multi_logloss: 0.618913
[256]	ddev's multi_logloss: 0.590558	dval's multi_logloss: 0.61803
[257]	ddev's multi_logloss: 0.589581	dval's multi_logloss: 0.617134
[258]	ddev's multi_logloss: 0.58854	dval's multi_log

[365]	ddev's multi_logloss: 0.508602	dval's multi_logloss: 0.544356
[366]	ddev's multi_logloss: 0.508014	dval's multi_logloss: 0.543853
[367]	ddev's multi_logloss: 0.507439	dval's multi_logloss: 0.543349
[368]	ddev's multi_logloss: 0.506882	dval's multi_logloss: 0.542866
[369]	ddev's multi_logloss: 0.506334	dval's multi_logloss: 0.542397
[370]	ddev's multi_logloss: 0.505835	dval's multi_logloss: 0.541967
[371]	ddev's multi_logloss: 0.505284	dval's multi_logloss: 0.541506
[372]	ddev's multi_logloss: 0.50472	dval's multi_logloss: 0.541028
[373]	ddev's multi_logloss: 0.504179	dval's multi_logloss: 0.540559
[374]	ddev's multi_logloss: 0.50362	dval's multi_logloss: 0.540073
[375]	ddev's multi_logloss: 0.503078	dval's multi_logloss: 0.539616
[376]	ddev's multi_logloss: 0.502523	dval's multi_logloss: 0.539153
[377]	ddev's multi_logloss: 0.501982	dval's multi_logloss: 0.538659
[378]	ddev's multi_logloss: 0.50146	dval's multi_logloss: 0.538208
[379]	ddev's multi_logloss: 0.500908	dval's multi_l

[486]	ddev's multi_logloss: 0.453383	dval's multi_logloss: 0.498542
[487]	ddev's multi_logloss: 0.452981	dval's multi_logloss: 0.498223
[488]	ddev's multi_logloss: 0.452626	dval's multi_logloss: 0.497957
[489]	ddev's multi_logloss: 0.452244	dval's multi_logloss: 0.497657
[490]	ddev's multi_logloss: 0.451852	dval's multi_logloss: 0.497355
[491]	ddev's multi_logloss: 0.451486	dval's multi_logloss: 0.497054
[492]	ddev's multi_logloss: 0.451122	dval's multi_logloss: 0.496773
[493]	ddev's multi_logloss: 0.45072	dval's multi_logloss: 0.496436
[494]	ddev's multi_logloss: 0.450349	dval's multi_logloss: 0.496154
[495]	ddev's multi_logloss: 0.449964	dval's multi_logloss: 0.49587
[496]	ddev's multi_logloss: 0.449623	dval's multi_logloss: 0.495586
[497]	ddev's multi_logloss: 0.449295	dval's multi_logloss: 0.495325
[498]	ddev's multi_logloss: 0.448933	dval's multi_logloss: 0.495041
[499]	ddev's multi_logloss: 0.448569	dval's multi_logloss: 0.494758
[500]	ddev's multi_logloss: 0.448224	dval's multi_

...

[1810]	ddev's multi_logloss: 0.252287	dval's multi_logloss: 0.378434
[1811]	ddev's multi_logloss: 0.252191	dval's multi_logloss: 0.378404
[1812]	ddev's multi_logloss: 0.252115	dval's multi_logloss: 0.378373
[1813]	ddev's multi_logloss: 0.252047	dval's multi_logloss: 0.378342
[1814]	ddev's multi_logloss: 0.251987	dval's multi_logloss: 0.37831
[1815]	ddev's multi_logloss: 0.25192	dval's multi_logloss: 0.378286
[1816]	ddev's multi_logloss: 0.251844	dval's multi_logloss: 0.378255
[1817]	ddev's multi_logloss: 0.251772	dval's multi_logloss: 0.37822
[1818]	ddev's multi_logloss: 0.2517	dval's multi_logloss: 0.378183
[1819]	ddev's multi_logloss: 0.25164	dval's multi_logloss: 0.378159
[1820]	ddev's multi_logloss: 0.251568	dval's multi_logloss: 0.378128
[1821]	ddev's multi_logloss: 0.251497	dval's multi_logloss: 0.378099
[1822]	ddev's multi_logloss: 0.251413	dval's multi_logloss: 0.378063
[1823]	ddev's multi_logloss: 0.251354	dval's multi_logloss: 0.378043
[1824]	ddev's multi_logloss: 0.25127	dva

[1930]	ddev's multi_logloss: 0.243807	dval's multi_logloss: 0.37521
[1931]	ddev's multi_logloss: 0.243732	dval's multi_logloss: 0.375186
[1932]	ddev's multi_logloss: 0.243671	dval's multi_logloss: 0.375162
[1933]	ddev's multi_logloss: 0.243595	dval's multi_logloss: 0.375126
[1934]	ddev's multi_logloss: 0.243538	dval's multi_logloss: 0.375105
[1935]	ddev's multi_logloss: 0.243481	dval's multi_logloss: 0.375077
[1936]	ddev's multi_logloss: 0.243419	dval's multi_logloss: 0.375057
[1937]	ddev's multi_logloss: 0.243356	dval's multi_logloss: 0.375029
[1938]	ddev's multi_logloss: 0.243275	dval's multi_logloss: 0.375008
[1939]	ddev's multi_logloss: 0.243193	dval's multi_logloss: 0.374973
[1940]	ddev's multi_logloss: 0.243118	dval's multi_logloss: 0.374947
[1941]	ddev's multi_logloss: 0.243041	dval's multi_logloss: 0.374915
[1942]	ddev's multi_logloss: 0.242965	dval's multi_logloss: 0.374885
[1943]	ddev's multi_logloss: 0.24291	dval's multi_logloss: 0.374858
[1944]	ddev's multi_logloss: 0.24284

[48]	ddev's multi_logloss: 1.44725	dval's multi_logloss: 1.66381
[49]	ddev's multi_logloss: 1.42956	dval's multi_logloss: 1.64698
[50]	ddev's multi_logloss: 1.41199	dval's multi_logloss: 1.63012
[51]	ddev's multi_logloss: 1.39535	dval's multi_logloss: 1.61383
[52]	ddev's multi_logloss: 1.3786	dval's multi_logloss: 1.59761
[53]	ddev's multi_logloss: 1.36216	dval's multi_logloss: 1.58172
[54]	ddev's multi_logloss: 1.34681	dval's multi_logloss: 1.56707
[55]	ddev's multi_logloss: 1.33093	dval's multi_logloss: 1.55174
[56]	ddev's multi_logloss: 1.31538	dval's multi_logloss: 1.53676
[57]	ddev's multi_logloss: 1.30006	dval's multi_logloss: 1.52193
[58]	ddev's multi_logloss: 1.28497	dval's multi_logloss: 1.50731
[59]	ddev's multi_logloss: 1.27013	dval's multi_logloss: 1.49297
[60]	ddev's multi_logloss: 1.25562	dval's multi_logloss: 1.47893
[61]	ddev's multi_logloss: 1.24142	dval's multi_logloss: 1.46524
[62]	ddev's multi_logloss: 1.22856	dval's multi_logloss: 1.45294
[63]	ddev's multi_logloss:

[171]	ddev's multi_logloss: 0.427642	dval's multi_logloss: 0.664965
[172]	ddev's multi_logloss: 0.423932	dval's multi_logloss: 0.661191
[173]	ddev's multi_logloss: 0.420243	dval's multi_logloss: 0.657457
[174]	ddev's multi_logloss: 0.416571	dval's multi_logloss: 0.653722
[175]	ddev's multi_logloss: 0.412926	dval's multi_logloss: 0.650032
[176]	ddev's multi_logloss: 0.409377	dval's multi_logloss: 0.646435
[177]	ddev's multi_logloss: 0.405786	dval's multi_logloss: 0.642812
[178]	ddev's multi_logloss: 0.402245	dval's multi_logloss: 0.639205
[179]	ddev's multi_logloss: 0.398757	dval's multi_logloss: 0.635661
[180]	ddev's multi_logloss: 0.395273	dval's multi_logloss: 0.632117
[181]	ddev's multi_logloss: 0.391845	dval's multi_logloss: 0.628636
[182]	ddev's multi_logloss: 0.388439	dval's multi_logloss: 0.625182
[183]	ddev's multi_logloss: 0.385082	dval's multi_logloss: 0.621761
[184]	ddev's multi_logloss: 0.381765	dval's multi_logloss: 0.618381
[185]	ddev's multi_logloss: 0.378487	dval's mult

KeyboardInterrupt: 

In [ ]:
beauty_result = pd.concat([beauty_result_dict[x] for x in beauty_result_dict.keys()])
fashion_result = pd.concat([fashion_result_dict[x] for x in fashion_result_dict.keys()])
mobile_result = pd.concat([mobile_result_dict[x] for x in mobile_result_dict.keys()])

In [ ]:
beauty_result.loc[beauty_result['tagging'] == '0', 'tagging'] = ''
fashion_result.loc[fashion_result['tagging'] == '0', 'tagging'] = ''
mobile_result.loc[mobile_result['tagging'] == '0', 'tagging'] = ''

In [ ]:
beauty_result.to_csv(os.path.join(result_path, 'beauty_result.csv'), index=False)
fashion_result.to_csv(os.path.join(result_path, 'fashion_result.csv'), index=False)
mobile_result.to_csv(os.path.join(result_path, 'mobile_result.csv'), index=False)